In [110]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
import nltk
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, accuracy_score,hamming_loss
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.base import BaseEstimator, TransformerMixin

[nltk_data] Downloading package punkt to C:\Users\zxc
[nltk_data]     ghoul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
import faiss

In [44]:
import torch
from transformers import BertTokenizer, BertModel

In [3]:
import pkg_resources

try:
    faiss_version = pkg_resources.get_distribution("faiss").version
    print(faiss_version)
except pkg_resources.DistributionNotFound:
    print("FAISS is not installed.")


1.7.4


In [4]:
db_name = 'recipe.db'
conn = sqlite3.connect(db_name)

table_name = 'recipe'

#забираем доступные категории
categories_sql = f"select c.id, c.name_cat from category c " \
                 f"join recipe_categories rc on " \
                 f"c.id = rc.cat_id " \
                 f"GROUP by c.id " \
                 f"HAVING count(*) > 100 " \
                 f"ORDER BY c.id"

loaded_categories = pd.read_sql(categories_sql, conn)

ids_cat = []
for i in range(len(loaded_categories)):
    ids_cat.append(str(loaded_categories.id[i]))

print(ids_cat)

#забираем способ приготовления рецепта с категориями из доступных с id категорий
sql = f"SELECT r.manual, (SELECT group_concat(rc.cat_id , ', ') " \
      f"from recipe_categories rc " \
      f"WHERE rc.recipe_id = r.id AND " \
      f"rc.cat_id in ({', '.join(ids_cat)})) " \
      f"AS categories from recipe r"

loaded_data = pd.read_sql(sql, conn)

#pandas data frame
conn.close()

['1', '2', '4', '5', '7', '8', '14', '16', '17', '20', '25', '27', '30', '33', '37', '43', '51', '62', '67', '68', '71', '90', '99', '103', '105', '139', '140', '150', '158', '257']


In [5]:
for i in range(len(ids_cat)):
    val = []
    for j in range(len(loaded_data)):
        spl = str(loaded_data.categories[j]).split(', ')
        #print(spl)
        #print(int(str(ids_cat[i]) in spl))
        val.append(int(str(ids_cat[i]) in spl))
    loaded_data.insert(loc=len(loaded_data.columns) , column=ids_cat[i], value=val)
#print(val)

In [17]:
loaded_data

,manual,categories,1,2,4,5,7,8,14,16,...,71,90,99,103,105,139,140,150,158,257
0,Шоколад разломать на кусочки и вместе со сливо...,"1, 2",1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Положите весь творог в кастрюльку и разомните ...,"4, 5",0,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Вскипятите воду в большой кастрюле и сварите п...,"7, 8",0,0,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Разогреть духовку. Отделить белки от желтков. ...,"1, 5",1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Взбить яйца с сахаром.\n\nПостепенно ввести му...,"1, 5, 14",1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6990,"Печенку посолить, поперчить, посыпать специями...","30, 5",0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6991,Крабовые палочки мелко нарезать или порубить в...,"17, 140",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6992,"Чечевицу промыть, залить водой на 4 см выше ур...",37,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6993,Нарезать филе средними полосками. Морковь наре...,"17, 67, 139",0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [6]:
#делим dataset на тренировочный и тестовый
X_train,X_test,y_train,y_test = train_test_split(loaded_data["manual"], loaded_data[ids_cat],test_size=0.3,random_state=42)

In [168]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    #print(sentence)
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [113]:
class Tokenizer(BaseEstimator, TransformerMixin):
    def __init__(self, remove_stop_words=True):
        self.remove_stop_words = remove_stop_words
        self.stop_words = set(stopwords.words('russian'))
        self.stemmer = SnowballStemmer('russian')

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return [self.tokenize_sentence(sentence) for sentence in X]

    def tokenize_sentence(self, sentence):
        tokens = word_tokenize(sentence, language="russian")
        tokens = [i for i in tokens if i not in string.punctuation]
        if self.remove_stop_words:
            tokens = [i for i in tokens if i not in self.stop_words]
        tokens = [self.stemmer.stem(i) for i in tokens]
        return tokens

In [171]:
class BertVectorizer(BaseEstimator, TransformerMixin):
    def __init__(self, model_name='cointegrated/rubert-tiny'):
        self.model_name = model_name
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertModel.from_pretrained(model_name)
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(self.device)
        self.model.eval()
        self.max_sequence_length = 510

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        vectors = []
        for tokens in X:
            tokens = tokens[:self.max_sequence_length]  # Ограничить длину последовательности до 512 токенов
            tokens = ['[CLS]'] + tokens + ['[SEP]']
            input_ids = self.tokenizer.convert_tokens_to_ids(tokens)
            input_tensor = torch.tensor(input_ids).unsqueeze(0).to(self.device)

            with torch.no_grad():
                outputs = self.model(input_tensor)
                embeddings = outputs[0]

            text_vector = embeddings.squeeze().mean(dim=0).cpu().numpy()
            vectors.append(text_vector)

        return vectors

In [190]:
class BertVectorizer2(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.tokenizer = BertTokenizer.from_pretrained('cointegrated/rubert-tiny')
        self.model = BertModel.from_pretrained('cointegrated/rubert-tiny')
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        vectors = []
        for text in X:
            text_vector = embed_bert_cls(text, model=self.model, tokenizer=self.tokenizer)
            vectors.append(text_vector)

        return vectors

In [191]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model = AutoModel.from_pretrained("cointegrated/rubert-tiny")
# model.cuda()  # uncomment it if you have a GPU

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings[0].cpu().numpy()

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [200]:
print(BertVectorizer2().transform(["привет мир", "awfawf"]))

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[array([-2.84503587e-02,  2.48572547e-02,  4.52326350e-02, -7.95979351e-02,
       -1.53341535e-02,  1.95008032e-02,  6.84027076e-02, -2.58912821e-03,
       -8.90688002e-02, -7.47816861e-02,  5.33769019e-02,  3.95353809e-02,
        2.15761568e-02,  1.92577671e-02,  8.65388010e-03,  1.72012746e-02,
        5.63668571e-02,  1.71543527e-02,  2.38393825e-02,  1.00879081e-01,
       -3.04934266e-03, -1.82165317e-02,  1.61965471e-02,  1.45853544e-02,
        8.97004232e-02,  2.11753529e-02, -1.13322604e-02,  1.37898242e-02,
       -1.44742774e-02,  5.83038032e-02, -9.02313739e-02, -7.45823830e-02,
        1.09646395e-01,  2.36695185e-02, -2.57688314e-02,  8.15490335e-02,
        1.42579060e-03, -1.82034243e-02, -1.73426896e-01, -3.17930169e-02,
       -7.68856108e-02,  8.63788426e-02,  1.19000688e-01,  5.18519012e-03,
       -2.13312805e-02,  4.75826524e-02,  3.57612483e-02, -1.77896786e-02,
        7.05813840e-02, -6.07864782e-02,  2.48750187e-02, -5.70905954e-02,
        1.31985229e-02, 

In [178]:
tokenizer = Tokenizer()
print(X_train[0])
tokens = [tokenizer.tokenize_sentence(X_train[0])]

Шоколад разломать на кусочки и вместе со сливочным маслом растопить на водяной бане, не переставая все время помешивать лопаткой или деревянной ложкой. Получившийся густой шоколадный соус снять с водяной бани и оставить остывать.

Тем временем смешать яйца со ста граммами коричневого сахара: яйца разбить в отдельную миску и взбить, постепенно добавляя сахар. Взбивать можно при помощи миксера или вручную — как больше нравится, — но не меньше двух с половиной-трех минут.

Острым ножом на разделочной доске порубить грецкие орехи. Предварительно их можно поджарить на сухой сковороде до появления аромата, но это необязательная опция.

В остывший растопленный со сливочным маслом шоколад аккуратно добавить оставшийся сахар, затем муку и измельченные орехи и все хорошо перемешать венчиком.

Затем влить сахарно-яичную смесь и тщательно смешать с шоколадной массой. Цвет у теста должен получиться равномерным, без разводов.

Разогреть духовку до 200 градусов. Дно небольшой глубокой огнеупорной фор

In [145]:
print(tokens)

[['шоколад', 'разлома', 'кусочк', 'вмест', 'сливочн', 'масл', 'растоп', 'водян', 'бан', 'перестав', 'врем', 'помешива', 'лопатк', 'деревя', 'ложк', 'получ', 'густ', 'шоколадн', 'соус', 'снят', 'водян', 'бан', 'остав', 'остыва', 'тем', 'времен', 'смеша', 'яйц', 'ста', 'грамм', 'коричнев', 'сахар', 'яйц', 'разб', 'отдельн', 'миск', 'взбит', 'постепен', 'добавл', 'сахар', 'взбива', 'помощ', 'миксер', 'вручн', '—', 'нрав', '—', 'меньш', 'двух', 'половиной-трех', 'минут', 'остр', 'нож', 'разделочн', 'доск', 'поруб', 'грецк', 'орех', 'предварительн', 'поджар', 'сух', 'сковород', 'появлен', 'аромат', 'эт', 'необязательн', 'опц', 'в', 'ост', 'растоплен', 'сливочн', 'масл', 'шоколад', 'аккуратн', 'добав', 'оста', 'сахар', 'зат', 'мук', 'измельчен', 'орех', 'перемеша', 'венчик', 'зат', 'влит', 'сахарно-яичн', 'сме', 'тщательн', 'смеша', 'шоколадн', 'масс', 'цвет', 'тест', 'долж', 'получ', 'равномерн', 'развод', 'разогрет', 'духовк', '200', 'градус', 'дно', 'небольш', 'глубок', 'огнеупорн', 'форм

In [158]:
vectorizer = BertVectorizer()
vector = vectorizer.transform(tokens)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


158


In [152]:
vector[0]

array([ 5.57968393e-02,  6.69642329e-01, -1.00083530e-01,  3.22767824e-01,
        5.94864130e-01, -2.67580420e-01,  1.91680398e-02, -4.65728849e-01,
       -6.62021279e-01,  3.78983200e-01,  2.87605792e-01, -4.56742615e-01,
       -1.90744549e-01, -1.88022584e-01, -1.61381841e-01,  6.41829222e-02,
        5.47489047e-01, -3.07966113e-01,  2.97192991e-01,  4.78653342e-01,
       -4.10718381e-01,  1.22976698e-01, -2.92649895e-01, -1.15826724e-05,
        5.53830683e-01,  3.95425975e-01,  1.73098341e-01,  5.37239134e-01,
        2.21128494e-01,  6.37348294e-01,  3.17309469e-01,  6.65508151e-01,
        3.98610353e-01,  1.49739575e+00, -2.13135526e-01, -5.88761270e-01,
       -9.27844048e-02, -1.42174855e-01, -1.19628347e-01,  1.65554717e-01,
       -4.98822153e-01,  8.94496441e-01,  3.16652626e-01,  2.89316237e-01,
        1.74797297e-01, -1.02147865e+00, -4.69532669e-01,  4.51981843e-01,
       -3.93779039e-01,  5.54213107e-01,  3.86881083e-02, -1.36399639e+00,
        7.02639446e-02, -

In [201]:
knn = Pipeline([
    ("vectorizer", BertVectorizer2()),
    ("normalizer", Normalizer()),
    ("model",  KNeighborsClassifier())
]
)
knn.fit(X_train, y_train)
accuracy_score(y_test, knn.predict(X_test))

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


0.05955216769890424

In [203]:
knn = Pipeline([
    ("vectorizer", TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True), max_features=768)),
    ("normalizer", Normalizer()),
    ("model",  KNeighborsClassifier())
]
)
knn.fit(X_train, y_train)
accuracy_score(y_test, knn.predict(X_test))

C:\Users\zxc ghoul\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.16055264411624584

In [ ]:
knn = Pipeline([
    ("vectorizer", BertVectorizer()),
    ("normalizer", Normalizer()),
    ("model",  KNeighborsClassifier())
]
)
knn.fit(X_train, y_train)
accuracy_score(y_test, knn.predict(X_test))

In [142]:
accuracy = knn.score(X_test, y_test)
print(accuracy)

# creating a confusion matrix
knn_predictions = knn.predict(X_test) #0.16055264411624584
#cm = confusion_matrix(y_test, knn_predictions)

0.03620771796093378


KeyboardInterrupt: 

In [169]:
class FaissKNeighbors:
    def __init__(self, k=5):
        self.index = None
        self.y = None
        self.k = k
        self.vectorizer = TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True))
        self.normalizer = Normalizer()

    def fit(self, X, y):
        X = self.vectorizer.fit_transform(X)
        X = self.normalizer.fit_transform(X)
        X = X.toarray().astype(np.float32)

        dimension = X.shape[1]
        self.index = faiss.IndexFlatL2(dimension)
        self.y = y

        for i in range(X.shape[0]):
            x = X[i].astype(np.float32)
            self.index.add(x)

        self.y = y

    def predict(self, X):
        X = self.vectorizer.transform(X)
        X = self.normalizer.transform(X)
        X = X.toarray().astype(np.float32)  # Преобразование разреженной матрицы в массив плотных векторов
        distances, indices = self.index.search(X, k=self.k)
        votes = self.y[indices]
        predictions = np.array([np.argmax(np.bincount(x)) for x in votes])
        return predictions

In [170]:
knn_faiss = Pipeline([
    ("model", FaissKNeighbors())
])

knn_faiss.fit(X_train, y_train)
#accuracy_score(y_test, knn_faiss.predict(X_test))

C:\Users\zxc ghoul\AppData\Roaming\Python\Python310\site-packages\sklearn\feature_extraction\text.py:525: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


ValueError: not enough values to unpack (expected 2, got 1)

In [43]:
knn_faiss.predict(X_test)

KeyError: "None of [Index([(4528, 3823, 4125, 2848, 4070), (1022, 2780, 2971, 4207, 3028),\n         (2821, 15, 2662, 1171, 4036), (1771, 1158, 3840, 2365, 1461),\n       (2030, 1937, 3960, 3235, 1880),  (3940, 2678, 462, 4371, 4356),\n       (1675, 2913, 3365, 1759, 4545), (3702, 4029, 2714, 4108, 3992),\n         (1776, 775, 2174, 1017, 652),   (1881, 3568, 2155, 71, 2140),\n       ...\n        (4793, 3721, 2729, 2075, 651),  (2246, 3161, 261, 4818, 3039),\n         (2259, 565, 266, 3447, 1438),  (4004, 207, 3885, 2165, 1725),\n       (2064, 2550, 2450, 3502, 4522),   (51, 2954, 4887, 2997, 4802),\n         (3287, 2024, 1369, 3196, 38),    (2899, 3047, 242, 50, 2561),\n         (918, 4021, 3874, 3889, 155),  (2999, 3478, 4396, 147, 4458)],\n      dtype='object', length=2099)] are in the [columns]"